## PID using the beam monitors
This code is the default way of identifying particles using the beam monitor. It is providing only a template event selection that is not optimised for any Physics analyses. It should serve as an exmple around which to build your own beam particle identification code. 

In [1]:
#Step 0, import libraries
import numpy as np
import importlib
#this is the file with the necessary functions for performing PID 
import beam_monitors_pid as bm


In [2]:
#Step 1, read in the data 

#### Example 1: medium momentum negative polarity
# run_number = 1478
# run_momentum = -410
# n_eveto_group = 1.01 #refractive index of ACT0-2
# n_tagger_group = 1.06 # of ACT3-5
# there_is_ACT5 = False  #important to keep track of whether there is ACT5 in the run  


### Example 2: relatively high momentum, positive polarity
run_number = 1610
run_momentum = 760
n_eveto_group = 1.01
n_tagger_group = 1.015
there_is_ACT5 = True

###Example 3: low momentum, positive polarity 
# run_number = 1308
# run_momentum = 220
# n_eveto_group = 1.01
# n_tagger_group = 1.15
# there_is_ACT5 = False

#choose the number of events to read in, set to -1 if you want to read all events
n_events = -1 #100000

#Set up a beam analysis class 
ana = bm.BeamAnalysis(run_number, run_momentum, n_eveto_group, n_tagger_group, there_is_ACT5)

#Store into memory the number of events desired
ana.open_file(n_events)

Reading in events: 100%|██████████| 1136955/1136955 [20:30<00:00, 924.18evt/s]

Total weight of self.df: 224.12Mb


In [3]:
#Step 2: Adjust the 1pe calibration: need to check the accuracy on the plots
# which are stored in plots/PID_run{run_number}_p{run_momentum}.pdf
ana.adjust_1pe_calibration()

One PE calibration finished, please don't forget to check that it is correct


In [4]:
#Step 3: proton and heavier particle tagging with T0-T1 TOF
#We need to tag protons before any other particles to avoid double-counting
ana.tag_protons_TOF()
#TODO: identify protons that produce knock-on electrons 

A total of 426286 protons and 3155 deuterons nuclei are tagged using the TOF out of 1124422, i.e. 37.9% of the dataset are protons and 0.3% are deuteron


In [5]:
#Step 4: tag electrons using ACT0-2 finding the minimum in the cut line
ana.tag_electrons_ACT02()

#Question: should we apply an additional electron tagging using ACT35? 
#to ensure that all has been removed? Not in the template selection but potentially in your own selection

A total of 105568 electrons are tagged with ACT02 out of 1124422, i.e. 9.4% of the dataset


In [6]:
#Step 5: check visually that the electron and proton removal makes sense in ACT35
ana.plot_ACT35_left_vs_right()

ACT35 left vs right plots have been made please check that they are sensible, electrons should be in the top right corner
ACT35 left vs right plots have been made please check that they are sensible, protons should be in the bottom left corner


In [7]:
#Step 6: make the muon/pion separation, using the muon tagger in case 
#at least 0.5% of muons and pions are above the cut line. This is necessary in case the 
#Number of particles is too high to clearly see a minimum between the muons and pions
#A more thorough analysis might want to remove events that are close to the cut line for a higher purity
ana.tag_muons_pions_ACT35()


The muon tagger charge has been plotted. The optimal cut line is at 192.1 a.u., there are 5018 electrons above the cut line and 5247 muons and pions, i.e. 0.9% of all muons and pions (589413)...
there are more than 0.5% of muons and pions above the muon tagger cut, we are applying it. (Please verify this on the plots)
ACT35 left vs right plots have been made please check that they are sensible, electrons should be in the top right corner
ACT35 left vs right plots have been made please check that they are sensible, protons should be in the bottom left corner
The pion/muon separtion cut line in ACT35 is 3.6, out of 589413.0 pions and muons, 9.5% are muons and 90.5 are pions


In [8]:
#Step 7: estimate the momentum for each particle from the T0-T1 TOF
#Note: we will save:
# 1. the momentum as the particle escapes the beam pipe and its error
# 2. the momentum as the particle escapes the WCTE beam window and its error
# 3. the mean momentum for this particle type and the associated error 

# first measure the particle TOF, make the plot
#This corrects any offset in the TOF (e.g. from cable length) that can cause the TOF 
#of electrons to be different from L/c This has to be calibrated to give meaningful momentum 
#estimates later on
ana.measure_particle_TOF()


The time difference between the reconstructed electron TOF and L/c = 14.81 is -0.86 ns


In [9]:
#This function extimates both the mean momentum for each particle type and for each trigger
#We take the the error on the tof for each trigger is the resolution of the TS0-TS1 measurement
#Taken as the std of the gaussian fit to the electron TOF
#This is still a somewhat coarse way of estimating uncertainty... 
#This also saves the momentum after exiting the beam window, recosntructed using the same techinque
#Final momentum is after exiting through the beam pipe
ana.estimate_momentum()

#note, because of fluctuations in the TOF, the reconstructed momentum will be unphysical for 
#some fo the events on the faster side of the distribution, this means that the 
#distibution of momenta event by event will be non-symetrical with a tail at low momenta
#The mean momentum will be fine though 

pion_mom [ 444.90327186  480.72750121  516.53385137  552.32409168  588.09558722
  623.85991257  659.60717425  695.37336351  731.11943588  766.86295837
  802.58914952  838.34180096  874.06218929  909.80754121  945.54288352
  981.27557557 1017.01205612 1052.75526745]
The estimated mean particle momenta are {'electron': 0, 'muon': 649.9275406373691, 'pion': 739.6942444198498, 'proton': 747.6417723031836, 'deuteron': 736.5031224496774} MeV/c with an error {'electron': 0, 'muon': 1.3886340788200187, 'pion': 0.3186019566342111, 'proton': 0.026507614707611538, 'deuteron': 0.33668656450606704} MeV/c


/eos/home-i01/a/acraplet/analysis_tools/notebooks/beam_monitors_pid.py:1689: RuntimeWarning: invalid value encountered in divide
  intercept = (y_b * x_a - y_a * x_b) / (x_a - x_b)
/eos/home-i01/a/acraplet/analysis_tools/notebooks/beam_monitors_pid.py:1689: RuntimeWarning: invalid value encountered in divide
  intercept = (y_b * x_a - y_a * x_b) / (x_a - x_b)
/eos/home-i01/a/acraplet/analysis_tools/notebooks/beam_monitors_pid.py:1689: RuntimeWarning: invalid value encountered in divide
  intercept = (y_b * x_a - y_a * x_b) / (x_a - x_b)
/eos/home-i01/a/acraplet/analysis_tools/notebooks/beam_monitors_pid.py:1689: RuntimeWarning: invalid value encountered in divide
  intercept = (y_b * x_a - y_a * x_b) / (x_a - x_b)
/eos/home-i01/a/acraplet/analysis_tools/notebooks/beam_monitors_pid.py:1689: RuntimeWarning: invalid value encountered in divide
  intercept = (y_b * x_a - y_a * x_b) / (x_a - x_b)
/eos/home-i01/a/acraplet/analysis_tools/notebooks/beam_monitors_pid.py:1689: RuntimeWarning: in


 
      act0_l    act1_l    act2_l    act3_l    act4_l    act5_l    act0_r  \
0  0.005390 -0.001362  0.023694  0.035741  0.048054  0.074607  0.117766   
1 -0.016688 -0.040814 -0.028617 -0.005654  0.621634 -0.027079  0.039705   
2 -0.060843 -0.050676  0.004077  0.004695 -0.047543 -0.008013  0.000674   
3 -0.038765 -0.040814 -0.041695 -0.036701  0.012205  0.023764  0.078736   
4  0.027468 -0.030951  0.030233  0.009869  0.878551 -0.014368  0.176312   

     act1_r    act2_r    act3_r  ...  is_proton  is_deuteron  is_electron  \
0  0.821982  0.010051  0.065365  ...      False        False        False   
1 -0.044000 -0.024899  1.191872  ...       True        False        False   
2  0.004110  0.038010  0.041498  ...      False        False        False   
3 -0.024756  0.010051  0.003311  ...      False        False        False   
4  0.032976 -0.017909  0.003311  ...      False        False        False   

   is_muon  is_pion   tof_corr  initial_momentum  initial_momentum_error  \
0    F

In [10]:
#Visually, it looks like all the particles reach the TOF
ana.plot_TOF_charge_distribution()


In [11]:
#Step X: end_analysis, necessary to cleanly close files 
ana.end_analysis()

In [ ]:
#Output to a root file
ana.output_beam_ana_to_root()

### List of the relevant beam PID information saved to the output file
After the beam PID has been performed we are saving the relevant variables to an output file, ideally root, to compare with other reconstructions:

##### Branch: beam_analysis

1. ACT i right (charge in ACT i right PMT, in units of PE) 
2. ACT i left  (for precise selections)
3. ACT02 total (for coarse selections)
4. ACT35 total 
5. T0-T1 TOF (called "tof")
6. T0-T4 TOF #this still has bugs I think
7. T4-T1 TOF  #this still has bugs I think
8. Muon tagger information (left and right)

16. Estimated PID from the beam information #todo, have a likelihood for each particle type
10. Estimated initial momenta for each trigger and error on mean
10. Estimated momenta exiting the beampipe for each trigger and error
17. Whether event passes beam data quality cuts
18. Total TOF detector charge, one can add a selection cut to remove events which do not cross the TOF for further analysis
19. ref0 and ref1 times (reference times of each digitiser) 

##### Branch: run_info
14. run number
13. run momentum (nominal, from CERN)
14. Aerogel refrective index information
15. Whether ACT5 is in the beamline

##### Branch: scalar_results

18. Position of each cut line and whether we apply the muon tagger cut for pion/muon separation (see step 6)
9. Mean tof for each particle type and error on mean
9. Mean momenta for each particle type, gaussian std and error on mean



/eos/home-i01/a/acraplet/analysis_tools/notebooks/beam_monitors_pid.py:2370: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[False  True False ...  True  True  True]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  self.df_all.loc[self.df_all["is_kept"], col] = self.df[col].values
/eos/home-i01/a/acraplet/analysis_tools/notebooks/beam_monitors_pid.py:2370: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[False False False ... False False False]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  self.df_all.loc[self.df_all["is_kept"], col] = self.df[col].values
/eos/home-i01/a/acraplet/analysis_tools/notebooks/beam_monitors_pid.py:2370: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future versio

Saved output file to beam_analysis_output_R1610.root


#### This is the end of the analysis please check the plots on plots/PID_run{run_number}\_p{run_momentum}.pdf and the analysis output in beam_analysis_output_R{run_number}.root